# In-class exercise 7

October 7, 2019

INFO 5871-001

## Imports

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import openpyxl
import re

## Load Excel data

Setting `read_only=True` makes it more efficient, because otherwise Python tries to preserve the data in a form that can be written back to Excel. Setting `data_only=True` gives us the data that you see when you look at the spreadsheet, rather than any underlying formulas.

In [2]:
wb = openpyxl.load_workbook('Regional.xlsx', read_only=True, data_only=True)

### Get the sheets for the different regions

There are 21 Health Service Regions. Here we extract them using a regular expression.

### get_entry

A general purpose function to get data from a specific spreadsheet location. We need this because we are picking and choosing individual data elements, rather than whole spreadsheet ranges.

In [3]:
def get_entry(wb, sheetname, row, col):
    """
    Returns the content of a spreadsheet element.
    
    Arguments:
    wb -- Excel workbook object from openpyxl
    sheetname -- The name of the sheet to extract from
    row, col -- The row and column where the data is to be extracted
    """
    sheet = wb[sheetname]
    cell = f'{col}{row}'
    return sheet[cell].value

### Helper function

Gets the data associated with specific years and specific questions. This makes it easier to add new questions if data extraction needs change.

In [4]:
quest_table = {'Uninsured': 15,'Public Ins': 12, 'Doc12': 20, 'Oral Health': 69, 
               'Med Cost': 85, 'Doc Cost': 86, 'Mental Health': 72, 'Overall Health': 78}
year_table = {2011: 'E', 2013: 'G', 2015: 'I', 2017: 'K'}



Example call of the function

In [5]:
#get_question_value(wb, 'HSR 1', 'Uninsured', 2013)

### Create the data frame

Steps:
- Initialize dictionary that we will use to build the data frame
- Build columns for year and region information
- Add to dictionary
- For each question, extract the question data into a list and add this to the dictionary
- Create a data frame with the dictionary as input

Use `describe` to see the overall shape of the data.

The different number of columns indicates that some weren't processed as numeric. Correct by explicit casting to `float`.

Create a MultiIndex with region and year. Rows can now be indexed with the combination of region and year. 

Now we're ready to do a `pairplot`. 

What correlations and anti-correlations seem evident?

## Denver metro

Let's pull out just the data for the Denver metro area, which contains regions 17, 21, 3, 16, 20. 

In [6]:
metro_regions = [17, 21, 3, 16, 20]


Because it is the index, we don't have to do any boolean indexing. We can just supply the region names.

## Question 3: Doctor vs Medicine Costs


It seems that more people are concerned about the costs of doctors than the cost of medicine. Is this a statistically significant difference?

Assumption: 10,000 households were chosen in Colorado. We will assume these are evenly distributed by regional population. We will also assume that the state officials did a good job and chose people randomly.

- Parameter of interest: difference in proportions $\hat{p_1} - \hat{p_2}$
- H0: The difference is zero. No difference in the groups.
- H1: There is a significant difference
- Significance level 95%, $\alpha$ = 0.05
- $n_1$ = Metro population/Total population * 10,000
- $n_2$ = same because it is the same population
- $SE = \sqrt{\frac{p_1(1-p_1)}{n_1} + \frac{p_2(1-p_2)}{n_2}}$
- $\alpha$ = 0.05
- two-tailed test: if the confidence interval for the difference of means excludes 0, then the two means are significantly different.

Total population of the state in 2017. L8 + L15

We are also going to assume that the fraction of the population in different areas didn't change since 2011. Probably unrealistic. J8 + J15

Bigger and more suspect assumption: we can ignore the size of each area when trying to create an overall mean from individual means. In the homework, we'll do this right. 

Conclusion

